In [ ]:
# !pip install PyPDF2 pdfminer.six pandas

  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)
   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   ------- -------------------------------- 1.0/5.6 MB 7.2 MB/s eta 0:00:01
   ------------------ --------------------- 2.6/5.6 MB 7.6 MB/s eta 0:00:01
   -------------------------- ------------- 3.7/5.6 MB 6.6 MB/s eta 0:00:01
   ---------------------------------------- 5.6/5.6 MB 7.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------- ----------------- 1.8/3.2 MB 10.1 MB/s eta 0:00:01
   ---------------------------------------- 3.2/3.2 MB 9.4 MB/s eta 0:00:00


In [5]:
import os
import glob
import json
import pickle
import PyPDF2
from pdfminer.high_level import extract_text
import pandas as pd

def convert_pdf_to_text(pdf_path, output_dir):
    """
    PDF 파일을 텍스트 파일로 변환
    PyPDF2로 처리가 안 되는 경우 pdfminer를 사용
    파일 이름은 '(' 문자 이전까지만 사용
    """
    # 원본 파일 이름에서 '(' 이전까지만 사용
    full_filename = os.path.splitext(os.path.basename(pdf_path))[0]
    # '(' 문자가 있는 경우 그 이전까지만 사용
    if '(' in full_filename:
        filename = full_filename.split('(')[0].strip()
    else:
        filename = full_filename
    
    output_text_path = os.path.join(output_dir, f"{filename}.txt")
    
    try:
        # 먼저 PyPDF2로 시도 (속도가 빠름)
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page_num in range(len(reader.pages)):
                page = reader.pages[page_num]
                text += page.extract_text() + "\n\n"
        
        # 추출된 텍스트가 너무 적거나 없으면 pdfminer 사용
        if len(text.strip()) < 100:
            text = extract_text(pdf_path)
    
    except Exception as e:
        print(f"PyPDF2 처리 중 오류 발생: {e}")
        # 백업 방법으로 pdfminer 사용 (한글 지원이 더 좋음)
        try:
            text = extract_text(pdf_path)
        except Exception as e:
            print(f"pdfminer 처리 중 오류 발생: {e}")
            text = f"오류: {pdf_path} 파일을 처리할 수 없습니다."
    
    # 텍스트 파일로 저장
    with open(output_text_path, 'w', encoding='utf-8') as f:
        f.write(text)
    
    return text, output_text_path, filename

def save_as_json(all_texts, output_dir):
    """
    모든 PDF 내용을 하나의 JSON 파일로 저장
    {'파일 이름': 텍스트 내용} 형식
    """
    json_path = os.path.join(output_dir, 'law_data.json')
    
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(all_texts, f, ensure_ascii=False, indent=2)
    
    return json_path

def save_as_pickle(all_texts, output_dir):
    """
    모든 PDF 내용을 하나의 Pickle 파일로 저장
    """
    pickle_path = os.path.join(output_dir, 'law_data.pkl')
    
    with open(pickle_path, 'wb') as f:
        pickle.dump(all_texts, f)
    
    return pickle_path

def process_all_pdfs_in_directory(data_dir='rawdata', output_dir='data'):
    """
    지정된 디렉토리의 모든 PDF 파일을 처리
    """
    # 출력 디렉토리가 없으면 생성
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # 모든 PDF 파일 찾기
    pdf_files = glob.glob(os.path.join(data_dir, '*.pdf'))
    
    results = []
    all_texts = {}  # 모든 텍스트를 저장할 딕셔너리: {파일이름: 내용}
    
    for pdf_file in pdf_files:
        print(f"처리 중: {pdf_file}")
        
        # PDF를 텍스트로 변환
        text, text_path, clean_filename = convert_pdf_to_text(pdf_file, output_dir)
        
        # 모든 텍스트 내용 저장
        all_texts[clean_filename] = text
        
        results.append({
            '원본파일': pdf_file,
            '텍스트파일': text_path,
        })
    
    # 모든 내용을 JSON 파일로 저장
    json_path = save_as_json(all_texts, output_dir)
    print(f"모든 텍스트가 {json_path} JSON 파일에 저장되었습니다.")
    
    # 모든 내용을 Pickle 파일로 저장
    pickle_path = save_as_pickle(all_texts, output_dir)
    print(f"모든 텍스트가 {pickle_path} Pickle 파일에 저장되었습니다.")
    
    # 처리 결과 요약 CSV 생성
    if results:
        pd.DataFrame(results).to_csv(os.path.join(output_dir, '처리결과_요약.csv'), 
                                    index=False, encoding='utf-8')
        print(f"총 {len(results)}개 파일 처리 완료. 결과는 {output_dir} 폴더에 저장되었습니다.")
    else:
        print(f"{data_dir} 폴더에 PDF 파일이 없습니다.")

if __name__ == "__main__":
    process_all_pdfs_in_directory()

처리 중: rawdata\가사소송법(법률)(제19354호)(20231019) (6).pdf
처리 중: rawdata\가족관계의 등록 등에 관한 규칙(대법원규칙)(제03140호)(20240719) (2).pdf
처리 중: rawdata\가족관계의 등록 등에 관한 법률(법률)(제19841호)(20241227) (2).pdf
처리 중: rawdata\개별소비세법 시행령(대통령령)(제35355호)(20250301) (2).pdf
처리 중: rawdata\건강가정기본법(법률)(제20417호)(20240927) (1).pdf
처리 중: rawdata\공공주택 특별법 시행령(대통령령)(제35059호)(20241210) (1).pdf
처리 중: rawdata\공무원 재해보상법 시행령(대통령령)(제35393호)(20250320) (1).pdf
처리 중: rawdata\공무원연금법 시행규칙(총리령)(제01489호)(20180921) (1).pdf
처리 중: rawdata\공무원연금법 시행령(대통령령)(제32529호)(20220909) (1).pdf
처리 중: rawdata\공무원연금법(법률)(제19513호)(20230630) (1).pdf
처리 중: rawdata\국가인권위원회법(법률)(제20558호)(20241203) (1).pdf
처리 중: rawdata\국민기초생활 보장법 시행령(대통령령)(제33858호)(20231116) (1).pdf
처리 중: rawdata\국민연금법 시행규칙(보건복지부령)(제00993호)(20240301) (1).pdf
처리 중: rawdata\국민연금법 시행령(대통령령)(제34163호)(20240301) (1).pdf
처리 중: rawdata\국민연금법(법률)(제20584호)(20241220) (1).pdf
처리 중: rawdata\국적법 시행령(대통령령)(제33082호)(20221220) (1).pdf
처리 중: rawdata\국제사법(법률)(제18670호)(20220705) (1).pdf
처리 중: rawdata\군인 재해보상법 시행규칙(국방부